
<div class="elfjS" data-track-load="description_content"><p>You are given an array of transactions <code>transactions</code> where <code>transactions[i] = [from<sub>i</sub>, to<sub>i</sub>, amount<sub>i</sub>]</code> indicates that the person with <code>ID = from<sub>i</sub></code> gave <code>amount<sub>i</sub> $</code> to the person with <code>ID = to<sub>i</sub></code>.</p>

<p>Return <em>the minimum number of transactions required to settle the debt</em>.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre><strong>Input:</strong> transactions = [[0,1,10],[2,0,5]]
<strong>Output:</strong> 2
<strong>Explanation:</strong>
Person #0 gave person #1 $10.
Person #2 gave person #0 $5.
Two transactions are needed. One way to settle the debt is person #1 pays person #0 and #2 $5 each.
</pre>

<p><strong class="example">Example 2:</strong></p>

<pre><strong>Input:</strong> transactions = [[0,1,10],[1,0,1],[1,2,5],[2,0,5]]
<strong>Output:</strong> 1
<strong>Explanation:</strong>
Person #0 gave person #1 $10.
Person #1 gave person #0 $1.
Person #1 gave person #2 $5.
Person #2 gave person #0 $5.
Therefore, person #1 only need to give person #0 $4, and all debt is settled.
</pre>

<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>1 &lt;= transactions.length &lt;= 8</code></li>
	<li><code>transactions[i].length == 3</code></li>
	<li><code>0 &lt;= from<sub>i</sub>, to<sub>i</sub> &lt; 12</code></li>
	<li><code>from<sub>i</sub> != to<sub>i</sub></code></li>
	<li><code>1 &lt;= amount<sub>i</sub> &lt;= 100</code></li>
</ul>
</div>


# Graph Traversal

This solution is not complete, but we write a graph traversal program that 1) simplifies cycles and 2) simplifies bidirectional edges. By doing so, we exploit simple redundancies in the money that has been transferred among people.

In [44]:
from typing import *
from collections import defaultdict

In [45]:
class Solution:
    def minTransfers(self, transactions: List[List[int]], verbose: bool = False) -> int:

        ## 1) build the graph

        graph = [] # each node has a unique idx in range [0, total nodes - 1]
        incoming_edges = []
        for i, transaction in enumerate(transactions):
            # node sent money, nnode received the money, amount is the money given
            node, nnode, amount = transaction
            g = len(graph)
            j = max(node, nnode)
            if j >= g:
                # Since we do not know the number of nodes apriori, we must extend
                # our graph length as we traverse the list
                # Default dict because if this is the first time node is pointing to nnode, its default
                # value should be 0. This helps with handle multiple transactions with the same pair
                # of senders and receivers
                ext_len = j - (g - 1)
                graph.extend([defaultdict(int) for _ in range(ext_len)])
                incoming_edges.extend([0]*ext_len)

            # form the edge
            e = graph[node]
            e[nnode] += amount
            # mark that nnode does have at least one incoming edge
            incoming_edges[nnode] += 1

        num_nodes = len(graph)

        if verbose:
            print("Step 1:")
            print(f"Number of nodes: {num_nodes}")
            print(f"Graph: {graph}")
            print(f"Incoming edges: {incoming_edges}")
            print("")
            print(f"Step 2:")

        ## 2) detect cycles and simplify bidirectional edges

        # stack should be initialized with nodes that have no incoming edges
        # each tuple in the stack is a tuple with the node id and its ancestors
        stack: List[Tuple[int, tuple]] = [(i, ()) for i, e in enumerate(incoming_edges) if e == 0]
        if len(stack) == 0:
            # All nodes have at least one incoming edge indicating that there must be a giant cycle.
            # Just push the first node
            stack = [(0, ())]
        if verbose:
            print(f"Initial Stack: {[s[0] for s in stack]}")

        visited = [False] * num_nodes # marks if a node has been visited during traversal
        cycles = set()
        while len(stack) > 0:
            node, ancestors = stack.pop()
            visited[node] = True
            node_edges = graph[node]

            for nnode in node_edges:
                # parse neighbors
                n_to_nn_exists = True # flag indicating if the edge node->nnode exists
                nnode_edges = graph[nnode]

                if not visited[nnode]:
                    # nnode has not been visited before

                    if node in nnode_edges:
                        # We just visited node, and nnode has not been visited, it may be that
                        # bidirectional edges exist. If nnode had been visited, this would have already
                        # been handled.
                        w_sum = node_edges[nnode] - nnode_edges[node]
                        if w_sum == 0:
                            # direct debt cancels out
                            del node_edges[nnode]
                            del nnode_edges[node]
                            n_to_nn_exists = False
                            incoming_edges[node] -= 1
                            incoming_edges[nnode] -= 1
                        elif w_sum > 0:
                            # node gave more money to nnode
                            node_edges[nnode] = w_sum
                            del nnode_edges[node]
                            incoming_edges[node] -= 1
                        else:
                            # nnode gave more money to node
                            del node_edges[nnode]
                            nnode_edges[node] = -w_sum
                            n_to_nn_exists = False
                            incoming_edges[nnode] -= 1

                    if n_to_nn_exists:
                        # The neighbor has not been visited, and the edge node->nnode
                        # still exists. Push nnode to the stack and add this node as the
                        # most recent ancestor.
                        stack.append((nnode, ancestors + (node,)))
                    elif incoming_edges[nnode] == 0:
                        # We have deleted the edge node->nnode but nnode has not been visited
                        # and now has no incoming edges. We must push it to the stack otherwise
                        # it will never be traversed
                        stack.append((nnode, ()))

                if visited[nnode] and n_to_nn_exists:
                    # we have visited nnode and our edge from node->nnode still exists,
                    # therefore we have a cycle
                    i = ancestors.index(nnode)
                    if verbose:
                        print(f"We have {node}->{nnode} and {nnode} is an ancestor. Ancestors: {ancestors[i:]}.")
                        print(f"Nodes in the cycle: {ancestors[i:] + (node,)}")
                    cycles.add(ancestors[i:] + (node,))

        ## 3) Process each cycle to simplify them
        if verbose:
            print("Step 3:")
            print(f"Number of cycles: {len(cycles)}. Cycles: {cycles}\n")
        for cycle in cycles:
            c_len = len(cycle) # number of nodes in the cycle
            # pos. balance if a node has excess money (thus must owe money),
            # neg. balance if a node is in debt and is therefore owed money
            balance = [0] * c_len
            for i, node in enumerate(cycle):
                j = (i + 1) % c_len
                nnode = cycle[j]
                amount = graph[node][nnode]
                balance[node] -= amount # decrease balance because node gave away money
                balance[nnode] += amount # increase balance because nnode received money

                # delete the edge prematurely
                del graph[node][nnode]

            if not all(b == 0 for b in balance):
                # the debt cancels out in this cycle, we don't need to add back any edges
                a, b, amount = None, None, None
                for i, b in enumerate(balance):
                    if b < 0:
                        a = i
                    if b > 0:
                        amount = b
                        b = i
                # Node a has negative balance because it gave the money to node b.
                # Create this edge.
                graph[a][b] += amount

        ## 4) Count transactions. This is the number of remaining edges in our simplified graph
        if verbose:
            # final (and simplified) graph
            print("Step 4:")
            print(f"Graph: {graph}\n")
        num_edges = sum([len(e) for e in graph])

        return num_edges

def main():
    test_cases = {
        "1": {
            "transactions": [[0,1,10],[2,0,5]],
            "expected": 2,
        },
        "2": {
            "transactions": [[0,1,10],[1,0,1],[1,2,5],[2,0,5]],
            "expected": 1,
        },
        "3": {
            "transactions": [[0,1,1],[1,2,1],[2,0,1]],
            "expected": 0,
        },
        "4": {
            "transactions": [[0,1,2],[1,2,1],[1,3,1]],
            "expected": 2,
        }
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        ret = solution.minTransfers(**targs, verbose=True)
        if expected is not None:
            passed = ret == expected
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}\n")
        print("---------------------")

main()


Step 1:
Number of nodes: 3
Graph: [defaultdict(<class 'int'>, {1: 10}), defaultdict(<class 'int'>, {}), defaultdict(<class 'int'>, {0: 5})]
Incoming edges: [1, 1, 0]

Step 2:
Initial Stack: [2]
Step 3:
Number of cycles: 0. Cycles: set()

Step 4:
Graph: [defaultdict(<class 'int'>, {1: 10}), defaultdict(<class 'int'>, {}), defaultdict(<class 'int'>, {0: 5})]

test case 1: {'transactions': [[0, 1, 10], [2, 0, 5]]}
Returned: 2, Expected: 2
Passed:True

---------------------
Step 1:
Number of nodes: 3
Graph: [defaultdict(<class 'int'>, {1: 10}), defaultdict(<class 'int'>, {0: 1, 2: 5}), defaultdict(<class 'int'>, {0: 5})]
Incoming edges: [2, 1, 1]

Step 2:
Initial Stack: [0]
We have 2->0 and 0 is an ancestor. Ancestors: (0, 1).
Nodes in the cycle: (0, 1, 2)
Step 3:
Number of cycles: 1. Cycles: {(0, 1, 2)}

Step 4:
Graph: [defaultdict(<class 'int'>, {0: 4}), defaultdict(<class 'int'>, {}), defaultdict(<class 'int'>, {})]

test case 2: {'transactions': [[0, 1, 10], [1, 0, 1], [1, 2, 5], [2, 0

# Leet Code Solution
Instead of using a graph, the balance of each person is summed. Dynamic programming is then used to determine the number of transactions.